In [1]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from sklearn.linear_model import LinearRegression
from scipy.stats import spearmanr

In [2]:
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py
!wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py

import feature_engineering

--2024-12-18 10:00:17--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/andrea_models.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3190 (3.1K) [text/plain]
Saving to: ‘andrea_models.py.1’

andrea_models.py.1  100%[===================>]   3.12K  --.-KB/s    in 0s      

2024-12-18 10:00:17 (37.8 MB/s) - ‘andrea_models.py.1’ saved [3190/3190]

--2024-12-18 10:00:18--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/course_library/feature_engineering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... conne

In [6]:
data_path = './data/'
df_X_train = pd.read_csv(data_path+'X_train.csv')#.drop(['COUNTRY'], axis=1).fillna(0) # given by the benchmark
df_X_test = pd.read_csv(data_path+'X_test.csv')
df_y_train = pd.read_csv(data_path+'Y_train.csv')

In [24]:
df_X_train[["ID", "DAY_ID", "COUNTRY"]]
df_X_train[["GAS_RET", "COAL_RET", "CARBON_RET"]]
df_X_train[["FR_TEMP", "FR_RAIN", "FR_WIND"]]
df_X_train[["FR_GAS", "FR_COAL", "FR_HYDRO", "FR_NUCLEAR", "FR_SOLAR", "FR_WINDPOW", "DE_LIGNITE"]]
df_X_train[["FR_CONSUMPTION", "FR_RESIDUAL_LOAD", "FR_NET_IMPORT", "FR_NET_EXPORT", "DE_FR_EXCHANGE"]]
df_y_train[["ID", "TARGET"]]

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733
...,...,...
1489,459,-0.172597
1490,1674,-0.063546
1491,748,0.151797
1492,1454,-0.640917


Search for constant feature

In [17]:
non_constants_features = feature_engineering.low_var_features(df_X_train, threshold=0)
print(f"number of features : {df_X_train.shape[1]}")
print(f"number of constant features : {df_X_train.shape[1] - len(non_constants_features)}")

number of features : 34
number of constant features : 0


Search for collinearity or quasi-collinearity

In [25]:
most_correlated_features = feature_engineering.get_most_correlated(df_X_train)
df_collinearity = most_correlated_features.reset_index()
df_collinearity.columns = ['feature 1', 'feature 2', 'correlation']
# df_collinearity.head(30)
set_to_remove_1 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD']
set_to_remove_2 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD', 'DE_WIND', 'DE_CONSUMPTION', 'DE_SOLAR', 'FR_CONSUMPTION']
set_to_remove_3 = ['DE_NET_IMPORT', 'FR_NET_IMPORT', 'DE_FR_EXCHANGE', 'FR_RESIDUAL_LOAD', 'DE_WIND', 'DE_CONSUMPTION', 'DE_SOLAR', 'FR_CONSUMPTION', 'DE_RESIDUAL_LOAD', 'DE_NET_EXPORT', 'DE_LIGNITE']
most_correlated_features

FR_NET_EXPORT   FR_NET_IMPORT      -1.000
DE_NET_EXPORT   DE_NET_IMPORT      -1.000
DE_FR_EXCHANGE  FR_DE_EXCHANGE     -1.000
FR_CONSUMPTION  FR_RESIDUAL_LOAD    0.965
DE_WIND         FR_WIND             0.821
                                    ...  
DAY_ID          DE_COAL            -0.001
DE_COAL         COAL_RET            0.001
DE_SOLAR        CARBON_RET          0.001
ID              FR_RAIN            -0.001
                COAL_RET           -0.000
Length: 561, dtype: float64

In [ ]:
most_correlated_features_v2

In [52]:
most_correlated_features_v2 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_1]])
col_selection_1 = [col for col in df_X_train.columns if col not in set_to_remove_1]
most_correlated_features_v2.head(15)

DE_WIND         FR_WIND             0.821
DE_CONSUMPTION  FR_CONSUMPTION      0.813
DE_SOLAR        FR_SOLAR            0.803
FR_CONSUMPTION  FR_GAS              0.780
DE_NET_EXPORT   DE_WINDPOW          0.731
DE_COAL         DE_LIGNITE          0.726
DE_GAS          DE_RESIDUAL_LOAD    0.715
DE_LIGNITE      DE_RESIDUAL_LOAD    0.714
DE_WINDPOW      DE_RESIDUAL_LOAD   -0.706
FR_CONSUMPTION  FR_NUCLEAR          0.706
                DE_SOLAR           -0.691
FR_DE_EXCHANGE  DE_NET_EXPORT      -0.686
DE_TEMP         FR_TEMP             0.680
DE_COAL         DE_RESIDUAL_LOAD    0.672
FR_DE_EXCHANGE  FR_NET_EXPORT       0.669
dtype: float64

In [53]:
most_correlated_features_v2 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_2]])
col_selection_2 = [col for col in df_X_train.columns if col not in set_to_remove_2]
most_correlated_features_v2.head(15)

DE_NET_EXPORT   DE_WINDPOW          0.731
DE_COAL         DE_LIGNITE          0.726
DE_GAS          DE_RESIDUAL_LOAD    0.715
DE_LIGNITE      DE_RESIDUAL_LOAD    0.714
DE_WINDPOW      DE_RESIDUAL_LOAD   -0.706
FR_DE_EXCHANGE  DE_NET_EXPORT      -0.686
DE_TEMP         FR_TEMP             0.680
DE_COAL         DE_RESIDUAL_LOAD    0.672
FR_DE_EXCHANGE  FR_NET_EXPORT       0.669
DE_NUCLEAR      FR_NUCLEAR          0.659
FR_NUCLEAR      FR_SOLAR           -0.648
DE_WINDPOW      FR_WINDPOW          0.637
DE_GAS          FR_GAS              0.604
DE_COAL         FR_COAL             0.598
FR_GAS          DE_COAL             0.539
dtype: float64

In [64]:
most_correlated_features_v3 = feature_engineering.get_most_correlated(df_X_train[[col for col in df_X_train.columns if col not in set_to_remove_3]])
col_selection_3 = [col for col in df_X_train.columns if col not in set_to_remove_3]
most_correlated_features_v3.head(15)

DE_TEMP         FR_TEMP          0.680
FR_DE_EXCHANGE  FR_NET_EXPORT    0.669
DE_NUCLEAR      FR_NUCLEAR       0.659
FR_NUCLEAR      FR_SOLAR        -0.648
DE_WINDPOW      FR_WINDPOW       0.637
DE_GAS          FR_GAS           0.604
DE_COAL         FR_COAL          0.598
FR_GAS          DE_COAL          0.539
FR_HYDRO        FR_NUCLEAR       0.533
DE_NUCLEAR      FR_SOLAR        -0.487
FR_DE_EXCHANGE  FR_GAS          -0.467
FR_NET_EXPORT   DE_NUCLEAR       0.458
DE_NUCLEAR      FR_WIND         -0.458
FR_GAS          FR_SOLAR        -0.456
FR_NET_EXPORT   FR_GAS          -0.454
dtype: float64

In [67]:
df_X_train_1 = df_X_train.set_index(['ID', 'DAY_ID'])
df_X_train_2 = df_X_train[col_selection_1].set_index(['ID', 'DAY_ID'])
df_X_train_3 = df_X_train[col_selection_2].set_index(['ID', 'DAY_ID'])
df_X_train_4 = df_X_train[col_selection_3].set_index(['ID', 'DAY_ID'])

In [56]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_1, Y_train_clean)

output_train = lr.predict(df_X_train_1)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.6%


In [57]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_2, Y_train_clean)

output_train = lr.predict(df_X_train_2)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.7%


In [58]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_3, Y_train_clean)

output_train = lr.predict(df_X_train_3)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 27.4%


In [68]:
lr = LinearRegression()

Y_train_clean = df_y_train['TARGET']

lr.fit(df_X_train_4, Y_train_clean)

output_train = lr.predict(df_X_train_4)

def metric_train(output):
    return  spearmanr(output, Y_train_clean).correlation

print('Spearman correlation for the train set: {:.1f}%'.format(100 * metric_train(output_train) ))

Spearman correlation for the train set: 24.2%


We obtain the best spearman's correlation with removing only a few features that have correlation with other variable higher than 0.95

<h3>CROSS-VALIDATION</h3>

In [72]:
from sklearn.model_selection import KFold

k_fold = KFold(n_splits=5, shuffle=True, random_state=6)

for i, (train_index, test_index) in enumerate(k_fold.split(df_X_train_2)):
    print(f"Fold {i}: ")
    print(f"  Train: index={train_index}")
    print(f"  Test: index={test_index}")

Fold 0: 
  Train: index=[   1    3    4 ... 1490 1491 1492]
  Test: index=[   0    2   11   12   14   15   21   24   33   37   40   50   55   56
   58   59   71   73   79   83   88   92   93   98  102  105  109  111
  117  132  136  137  140  148  150  152  155  168  171  178  179  186
  191  195  208  219  222  223  225  229  231  238  245  250  256  257
  261  263  264  268  286  287  290  291  302  304  307  313  329  336
  338  343  357  359  363  371  375  379  381  386  387  399  406  408
  421  430  432  433  434  436  438  439  444  463  465  477  478  486
  488  500  502  503  504  510  511  512  515  517  523  531  553  562
  563  568  570  581  588  592  595  600  602  605  607  608  610  616
  626  628  629  633  646  648  649  682  686  690  702  708  711  718
  733  735  737  739  741  751  752  756  757  761  764  765  767  771
  775  777  785  793  794  796  801  804  811  822  826  830  832  836
  843  851  855  865  874  875  877  883  885  888  894  902  904  905
  9

In [77]:
from sklearn.model_selection import cross_val_score

lr = LinearRegression()

scores = cross_val_score(lr, df_X_train_2, Y_train_clean, cv=k_fold)#, scoring='neg_mean_squared_log_error')
print(scores.mean())
print(scores.std())

0.00599742541935675
0.025925683319626934


Mean squared logarithmic error cannot be used when targets contain negative values. Same goes for Root mean squared logarithmic error